In [2]:
import pandas as pd
import numpy as np
import fasttext
import os
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from gensim.utils import simple_preprocess
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:

h = pd.read_csv("Airline-Sentiment-2-w-AA.csv")
h.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,airline_sentiment,airline_sentiment:confidence,negativereason,negativereason:confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,681448150,False,finalized,3,2/25/15 5:24,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2/24/15 11:35,5.703060e+17,NaN,Eastern Time (US & Canada)
1,681448153,False,finalized,3,2/25/15 1:53,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials to the experience... tacky.,NaN,2/24/15 11:15,5.703010e+17,NaN,Pacific Time (US & Canada)
2,681448156,False,finalized,3,2/25/15 10:01,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I need to take another trip!,NaN,2/24/15 11:15,5.703010e+17,Lets Play,Central Time (US & Canada)
3,681448158,False,finalized,3,2/25/15 3:05,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",NaN,2/24/15 11:15,5.703010e+17,NaN,Pacific Time (US & Canada)
4,681448159,False,finalized,3,2/25/15 5:50,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing about it,NaN,2/24/15 11:14,5.703010e+17,NaN,Pacific Time (US & Canada)


In [5]:
#looking for null values in the dataset
h.isnull()
#Since that a rating "0" means that the user hasn’t tried the recipe, I deleted that rating.
d = h[h.airline_sentiment != 0]
d

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,airline_sentiment,airline_sentiment:confidence,negativereason,negativereason:confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,681448150,False,finalized,3,2/25/15 5:24,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2/24/15 11:35,5.703060e+17,NaN,Eastern Time (US & Canada)
1,681448153,False,finalized,3,2/25/15 1:53,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials to the experience... tacky.,NaN,2/24/15 11:15,5.703010e+17,NaN,Pacific Time (US & Canada)
2,681448156,False,finalized,3,2/25/15 10:01,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I need to take another trip!,NaN,2/24/15 11:15,5.703010e+17,Lets Play,Central Time (US & Canada)
3,681448158,False,finalized,3,2/25/15 3:05,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",NaN,2/24/15 11:15,5.703010e+17,NaN,Pacific Time (US & Canada)
4,681448159,False,finalized,3,2/25/15 5:50,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing about it,NaN,2/24/15 11:14,5.703010e+17,NaN,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,681679794,False,finalized,3,2/25/15 19:46,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different flight to Chicago.,NaN,2/22/15 12:01,5.695880e+17,NaN,NaN
14636,681679795,False,finalized,3,2/25/15 19:14,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flight. No warnings or communication until we were 15 minutes Late Flight. That's called shitty customer svc,NaN,2/22/15 11:59,5.695870e+17,Texas,NaN
14637,681679796,False,finalized,3,2/25/15 19:04,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to #BlackBerry10,NaN,2/22/15 11:59,5.695870e+17,"Nigeria,lagos",NaN
14638,681679797,False,finalized,3,2/25/15 18:59,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my flight, and don't answer your phones! Any other suggestions so I can make my commitment??",NaN,2/22/15 11:59,5.695870e+17,New Jersey,Eastern Time (US & Canada)


In [37]:
#plotting total ratings
sns.set(rc = {'figure.figsize':(15,8)})
cx = sns.countplot(x="airline_sentiment", data=d)

NameError: name 'd' is not defined

In [8]:
#pre-proccessing the dataset
rc = pd.DataFrame(d,columns=['airline_sentiment','text'])
# Here we will be replacing rating numbers with the words for fastext
#rr['rating']=rr['rating'].replace({1 :'Terrible',2:'bad',3:'Neutral',4:'Good',5:'Excellent'})
#droping null values from the dataset
rc.dropna(subset = ["text"], inplace=True)
rc.dropna(subset = ["airline_sentiment"], inplace=True)
rc.to_csv('airline.csv')

In [9]:
# Importing the dataset and labelling the dataset for fasttext recognization 
dataset = pd.read_csv('airline.csv')[['text', 'airline_sentiment']].rename(columns = {'text': 'texts', 'airline_sentiment': 'airline_sentiments'})
#ds = pd.read_csv('valid.csv')[['Body', 'Y']].rename(columns = {'Body': 'questions', 'Y': 'category'})

# NLP Preprocess
dataset.iloc[:, 0] = dataset.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))
#ds.iloc[:, 0] = ds.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))

# Prefixing each row of the category column with '__label__'
dataset.iloc[:, 1] = dataset.iloc[:, 1].apply(lambda x: '__label__' + str(x))

In [11]:
dataset[['airline_sentiments', 'texts']].to_csv('Airline_Review.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

In [12]:
#Applying fasttext
!head -n 11640 Airline_Review.txt > airline.train
!tail -n 3000 Airline_Review.txt > airline.valid

In [21]:
model = fasttext.train_supervised(input="airline.train")
model.save_model("model_airline.bin")

In [22]:
model.predict("@VirginAmerica I didn't today... Must mean I need to take another trip!")

(('__label__neutral',), array([0.87785715]))

In [23]:
model.test("airline.valid")

(3000, 0.8216666666666667, 0.8216666666666667)

In [24]:
model.test("airline.valid", k=5)

(3000, 0.3333333333333333, 1.0)

In [25]:
#Pre-processing the dataset
!cat Airline_Review.txt  | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > Airline_Review.preprocessed.txt
!head -n 11640 Airline_Review.preprocessed.txt > airline_Pre.train
!tail -n 3000 Airline_Review.preprocessed.txt > airline_Pre.valid

In [26]:
model = fasttext.train_supervised(input="airline_Pre.train")
model.test("airline_Pre.valid")

(3000, 0.819, 0.819)

In [36]:
model = fasttext.train_supervised(input="airline_Pre.train", epoch=25)
model.test("airline_Pre.valid")

(3000, 0.796, 0.796)

In [35]:
model = fasttext.train_supervised(input="airline_Pre.train", lr=1.0)
model.test("airline_Pre.valid")

(3000, 0.8116666666666666, 0.8116666666666666)

In [33]:
model = fasttext.train_supervised(input="airline_Pre.train", lr=1.0, epoch=25)
model.test("airline_Pre.valid")

(3000, 0.7773333333333333, 0.7773333333333333)

In [34]:
model = fasttext.train_supervised(input="airline_Pre.train", lr=1.0, epoch=25, wordNgrams=2)
model.test("airline_Pre.valid")

(3000, 0.8176666666666667, 0.8176666666666667)

In [31]:
model.predict("I made the Mexican topping and took it to bunko. Everyone loved it.")

(('__label__positive',), array([0.98050201]))

In [76]:
#Applying multi-label
model = fasttext.train_supervised(input="airline_Pre.train", lr=0.5, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='ova')
model.test("airline_Pre.valid")
#model3.predict("This worked very well and is EASY. I used not quite a whole package (10oz) of white chips. Great!")

(3000, 0.7653333333333333, 0.7653333333333333)